# Wildfire Database Project

In [1]:
import pandas as pd

In [2]:
# define filepath of raw csv files
economic_file = 'raw_data/economic_data/economic-damage-from-natural-disasters.csv'
number_file = 'raw_data/economic_data/number-of-natural-disaster-events.csv'
fema_file = 'raw_data/fema_data/database.csv'

## Natural Disaster Economic Database

In [3]:
# import economic csv, isolate wildfire related data and drop unnecessary columns
economic_df = pd.read_csv(economic_file)
economic_df.head()

,Entity,Code,Year,Total economic damage from natural disasters (US$)
0,All natural disasters,NaN,1900,30000000
1,All natural disasters,NaN,1901,0
2,All natural disasters,NaN,1902,0
3,All natural disasters,NaN,1903,480000000
4,All natural disasters,NaN,1904,0


In [4]:
# import economic csv, isolate wildfire related data and drop unnecessary columns
economic_df = pd.read_csv(economic_file)
economic_df = economic_df.rename(columns = {'Total economic damage from natural disasters (US$)': 'Total_cost'})
economic_df = economic_df.loc[economic_df['Entity'] == 'Wildfire']
economic_df = economic_df.drop(columns = ['Code', 'Entity'])
economic_df.head()

,Year,Total_cost
517,1918,100000000
518,1922,8000000
519,1947,30000000
520,1965,1000000
521,1967,68500000


In [5]:
# economic_df.describe()

In [6]:
# import numbers csv, 
number_df = pd.read_csv(number_file)
number_df

,Entity,Code,Year,Number of reported natural disasters (reported disasters)
0,All natural disasters,NaN,1900,5
1,All natural disasters,NaN,1901,2
2,All natural disasters,NaN,1902,9
3,All natural disasters,NaN,1903,8
4,All natural disasters,NaN,1904,2
...,...,...,...,...
823,Wildfire,NaN,2014,4
824,Wildfire,NaN,2015,12
825,Wildfire,NaN,2016,10
826,Wildfire,NaN,2017,13


In [7]:
# locate wildfire data, and drop unnecessary columns
number_df = number_df.rename(columns = {'Number of reported natural disasters (reported disasters)'
: 'Total_Disasters'})
number_df = number_df.loc[number_df['Entity'] == 'Wildfire']
number_df = number_df.drop(columns = ['Code', 'Entity'])
number_df

,Year,Total_Disasters
766,1911,1
767,1918,1
768,1922,1
769,1929,1
770,1939,1
...,...,...
823,2014,4
824,2015,12
825,2016,10
826,2017,13


In [8]:
# merge number & yearly data into one dataframe with "Year" as a key
annual_data = pd.merge(economic_df, number_df, on='Year', how="outer")
# Sort "Year" to organize the data 
annual_data = annual_data.sort_values(["Year"])
annual_data

,Year,Total_cost,Total_Disasters
44,1911,NaN,1
0,1918,1.000000e+08,1
1,1922,8.000000e+06,1
45,1929,NaN,1
46,1939,NaN,1
...,...,...,...
39,2014,2.590000e+08,4
40,2015,3.439820e+09,12
41,2016,6.287000e+09,10
42,2017,1.019000e+09,13


In [9]:
# Change the columns' name for postgres
final_annual_data = annual_data.rename(columns = {"Year": "year", "Total_cost": "total_wildfire_cost", "Total_Disasters": "total_wildfire"})


In [10]:
# Reset the index number
final_annual_data = final_annual_data.reset_index(drop = True)
final_annual_data

,year,total_wildfire_cost,total_wildfire
0,1911,NaN,1
1,1918,1.000000e+08,1
2,1922,8.000000e+06,1
3,1929,NaN,1
4,1939,NaN,1
...,...,...,...
57,2014,2.590000e+08,4
58,2015,3.439820e+09,12
59,2016,6.287000e+09,10
60,2017,1.019000e+09,13


## FEMA Natural Disaster Dataset

In [11]:
# import fema csv, isolate all fire-related data, then drop unnecessary columns
fema_df = pd.read_csv(fema_file)
fema_df = fema_df.loc[fema_df['Disaster Type'] == 'Fire']
fema_df["Disaster Title"].unique()
fema_df = fema_df.drop(columns = ['Declaration Number', 'Declaration Type', 'Disaster Type', 'Disaster Title', 'Individual Assistance Program', 'Individuals & Households Program', 'Public Assistance Program', 'Hazard Mitigation Program'])
# split year & add column to dataframe
date_split = fema_df['Declaration Date'].str.split('/', n=2, expand = True)
fema_df['Year'] = date_split[2]
fema_df

,Declaration Date,State,County,Start Date,End Date,Close Date,Year
9,07/02/1953,NH,NaN,07/02/1953,07/02/1953,02/01/1956,1953
63,12/29/1956,CA,NaN,12/29/1956,12/29/1956,04/01/1959,1956
103,07/22/1960,ID,NaN,07/22/1960,07/22/1960,09/20/1961,1960
117,11/16/1961,CA,NaN,11/16/1961,11/16/1961,08/01/1963,1961
1001,08/30/1967,ID,Benewah County,08/30/1967,08/30/1967,06/17/1970,1967
...,...,...,...,...,...,...,...
44890,10/20/2015,WA,Pend Oreille County,08/09/2015,09/10/2015,NaN,2015
44891,10/20/2015,WA,Stevens County,08/09/2015,09/10/2015,NaN,2015
44892,10/20/2015,WA,Whatcom County,08/09/2015,09/10/2015,NaN,2015
44893,10/20/2015,WA,Yakima County,08/09/2015,09/10/2015,NaN,2015


In [12]:
# Change columns' name for Postgres
final_fema_df = fema_df.rename(columns = {"Declaration Date": "declaration_date", "State": "state", "County": "county", \
                                          "Start Date": "start_date", "End Date": "end_date", "Close Date": "close_date", 
                                          "Year": "year"})

In [13]:
# Reset index number
final_fema_df = final_fema_df.reset_index(drop=True)
final_fema_df

,declaration_date,state,county,start_date,end_date,close_date,year
0,07/02/1953,NH,NaN,07/02/1953,07/02/1953,02/01/1956,1953
1,12/29/1956,CA,NaN,12/29/1956,12/29/1956,04/01/1959,1956
2,07/22/1960,ID,NaN,07/22/1960,07/22/1960,09/20/1961,1960
3,11/16/1961,CA,NaN,11/16/1961,11/16/1961,08/01/1963,1961
4,08/30/1967,ID,Benewah County,08/30/1967,08/30/1967,06/17/1970,1967
...,...,...,...,...,...,...,...
2642,10/20/2015,WA,Pend Oreille County,08/09/2015,09/10/2015,NaN,2015
2643,10/20/2015,WA,Stevens County,08/09/2015,09/10/2015,NaN,2015
2644,10/20/2015,WA,Whatcom County,08/09/2015,09/10/2015,NaN,2015
2645,10/20/2015,WA,Yakima County,08/09/2015,09/10/2015,NaN,2015


In [15]:
# export files to csv
final_fema_df.to_csv('clean_data/final_fema.csv')
final_annual_data.to_csv('clean_data/final_yearly_data.csv')